<a href="https://colab.research.google.com/github/Safat23/Cse423-project-/blob/main/422_L2me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random  # We are using Python's 'random' module to generate random numbers

# ===============================
# STARTING SETUP (VARIABLES)
# ===============================

# This is how much money we are starting with in our simulation.
# Imagine you're going to do some trading, and you start with 1000 coins or dollars.
initial_capital = 1000

# This list represents how the price of a stock changed each day.
# Positive numbers = price went up, negative numbers = price went down.
# So -1.2 means the price dropped by 1.2%, and 3.4 means it went up by 3.4%
historical_prices = [-1.2, 3.4, -0.8, 2.1, -2.5, 1.7, -0.3, 5.8, -1.1, 3.5]

# Now we create some trading strategies. Think of these as little robots with rules for when to buy or sell.
# Each one has 3 parts:
# - stop_loss: if the market drops this much, stop trading to avoid losing more
# - take_profit: if the market goes up this much, sell and take the profit
# - trade_size: how much money they risk in each trade (as a percentage of capital)
initial_population = [
    {"stop_loss": 2, "take_profit": 5, "trade_size": 20},  # This strategy has a stop loss at 2%, profit at 5%, and uses 20% of money per trade
    {"stop_loss": 3, "take_profit": 7, "trade_size": 30},
    {"stop_loss": 1.5, "take_profit": 4, "trade_size": 25},
    {"stop_loss": 2.5, "take_profit": 6, "trade_size": 15}
]

# Number of times we want to improve (or evolve) the strategies
# You can think of each generation like a new batch of strategies, getting better and better
generations = 10

# ===============================
# HOW TO CHECK IF A STRATEGY IS GOOD
# ===============================

def calculate_fitness(strategy):
    # We start with some money
    capital = initial_capital

    # Go through each price change in history
    for price_change in historical_prices:
        # Decide how much money to trade based on trade_size
        # If trade_size is 20, and we have 1000, we trade 200
        trade_size = capital * (strategy["trade_size"] / 100)

        # If price goes down more than the stop_loss value, we lose money
        if price_change < -strategy["stop_loss"]:
            profit_or_loss = -trade_size * (strategy["stop_loss"] / 100)

        # If price goes up more than the take_profit value, we make profit
        elif price_change > strategy["take_profit"]:
            profit_or_loss = trade_size * (strategy["take_profit"] / 100)

        # Otherwise, we make or lose money based on actual price change
        else:
            profit_or_loss = trade_size * (price_change / 100)

        # Add or subtract the result from our total capital
        capital += profit_or_loss

    # At the end, we return how much profit we made in total
    return capital - initial_capital

# ===============================
# MIX TWO STRATEGIES TOGETHER (LIKE GENETIC PARENTS)
# ===============================

def crossover(parent1, parent2):
    # Pick a number (1 or 2) to decide what part to mix
    crossover_point = random.randint(1, 2)

    # Make copies of the parent strategies so we don’t change the original
    child1 = parent1.copy()
    child2 = parent2.copy()

    # If the number was 1, swap their take_profit values
    if crossover_point == 1:
        child1["take_profit"], child2["take_profit"] = child2["take_profit"], child1["take_profit"]
    else:
        # Otherwise, swap their trade_size values
        child1["trade_size"], child2["trade_size"] = child2["trade_size"], child1["trade_size"]

    return child1, child2  # Return the two new children

# ===============================
# RANDOMLY CHANGE A STRATEGY SLIGHTLY
# ===============================

def mutate(strategy):
    mutation_rate = 0.05  # This means we have a 5% chance to change each part

    # Maybe change the stop_loss
    if random.random() < mutation_rate:
        strategy["stop_loss"] = round(random.uniform(1, 10), 1)

    # Maybe change the take_profit
    if random.random() < mutation_rate:
        strategy["take_profit"] = round(random.uniform(1, 10), 1)

    # Maybe change the trade_size
    if random.random() < mutation_rate:
        strategy["trade_size"] = round(random.uniform(10, 50), 1)

    return strategy

# ===============================
# MAIN FUNCTION THAT RUNS THE WHOLE GENETIC ALGORITHM
# ===============================

def genetic_algorithm():
    population = initial_population.copy()  # Start with our original group of strategies

    for _ in range(generations):  # Do this for 10 generations (or whatever number we set)
        # Sort strategies by how good they are (highest fitness first)
        population = sorted(population, key=lambda x: -calculate_fitness(x))

        # Keep the top 2 strategies (the best ones)
        new_population = population[:2]

        # Fill the rest of the population with children made from these best parents
        while len(new_population) < len(population):
            parent1, parent2 = random.sample(population[:2], 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            if len(new_population) < len(population):
                new_population.append(mutate(child2))

        # Replace the old population with the new one
        population = new_population

    # At the end, find the best strategy in the final group
    best_strategy = max(population, key=calculate_fitness)
    final_profit = calculate_fitness(best_strategy)

    return best_strategy, final_profit

# ===============================
# RUN THE GENETIC ALGORITHM
# ===============================

best_strategy, final_profit = genetic_algorithm()  # Run the function and get the results

# Show the best strategy and how much money it made
print("best_strategy:", best_strategy)
print("Final_profit:", round(final_profit, 2))

# ===============================
# EXTRA: TWO-POINT CROSSOVER (SWAPS MORE THAN ONE PART)
# ===============================

def two_point_crossover():
    parent1, parent2 = random.sample(initial_population, 2)  # Pick any 2 parents randomly

    # Choose two points in the list of traits (0 to 2)
    points = sorted(random.sample(range(3), 2))

    # Make copies of the parents to create children
    child1, child2 = parent1.copy(), parent2.copy()

    # Get the list of trait names like ['stop_loss', 'take_profit', 'trade_size']
    for key in list(parent1.keys())[points[0]:points[1] + 1]:
        # Swap the values between child1 and child2 for these traits
        child1[key], child2[key] = child2[key], child1[key]

    # Show what happened
    print("\nParent 1:", parent1)
    print("Parent 2:", parent2)
    print("Child 1:", child1)
    print("Child 2:", child2)

# Run the example two-point crossover
two_point_crossover()


best_strategy: {'stop_loss': 3, 'take_profit': 7, 'trade_size': 30}
Final_profit: 31.97

Parent 1: {'stop_loss': 3, 'take_profit': 7, 'trade_size': 30}
Parent 2: {'stop_loss': 2.5, 'take_profit': 6, 'trade_size': 15}
Child 1: {'stop_loss': 3, 'take_profit': 6, 'trade_size': 15}
Child 2: {'stop_loss': 2.5, 'take_profit': 7, 'trade_size': 30}
